In [351]:
!pip install mesa

In [352]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import seaborn as sn

In [353]:
from google.colab import  files
uploaded = files.upload()

Saving mapa.txt to mapa (7).txt


In [354]:
class Cell():
  def __init__(self, x, y, wall):
    self.pos = (x, y)

    if wall[0] == '1': self.up = True
    else: self.up = False
    if wall[1] == '1': self.left = True
    else: self.left = False
    if wall[2] == '1': self.down = True
    else: self.down = False
    if wall[3] == '1': self.right = True
    else: self.right = False

    self.alert = 0 # 1- false alarm    2 - victim
    self.fire = 0 # 1 - smoke    2 - fire
    self.door = [] # pos of the connected cell with a door & if is open    [] if no door
    self.entrance = False # True if the current cell is a access point to the structure

In [355]:
class FiremanAgent(Agent):
    def __init__(self, unique_id, model, point=None):
      super().__init__(unique_id, model)
      self.point = point
      self.actionPoints = 4
      self.carryState = 1

    def step(self):
      myTurn = True
      if self.point is not None:
        path = self.dijkstra(self.pos, self.model.interestPoints[self.point].pos)[0]
        while(myTurn):
          if path and self.actionPoints >= self.calculateSteps(self.pos, path[0]):
            self.actionPoints -= self.calculateSteps(self.pos, path[0])
            self.model.grid.move_agent(self, path[0])
            path.pop(0)
          elif path and self.actionPoints < self.calculateSteps(self.pos, path[0]):
            if self.actionPoints + 4 > 8:
              self.actionPoints = 8
              myTurn = False
            else:
              self.actionPoints += 4
              myTurn = False
          if self.pos == self.model.interestPoints[self.point].pos:
            if self.model.interestPoints[self.point].alert == 2:
              self.model.interestPoints[self.point].alert = 0
              self.carryState = 2
              closestExit = None
              minDistance = 100
              for exit in self.model.outSide:
                distance = self.dijkstra(self.pos, exit.pos)[1]
                if distance < minDistance:
                  minDistance = distance
                  closestExit = exit
              self.model.cells[self.pos[0]][self.pos[1]].alert = 0
              self.model.interestPoints.append(closestExit)
              self.model.interestPoints[self.point] = self.model.generateNewInterestPoint()
              self.point = len(self.model.interestPoints) - 1
            elif self.model.cells[self.pos[0]][self.pos[1]] in self.model.outSide and self.carryState == 2:
              print("Vida salvada")
              self.carryState = 1
              self.model.savedLifes += 1
              self.model.interestPoints.remove(self.model.interestPoints[self.point])
            elif self.model.interestPoints[self.point].alert == 1:
              print("Falsa alarma")
              self.model.cells[self.pos[0]][self.pos[1]].alert = 0
              self.model.interestPoints[self.point] = self.model.generateNewInterestPoint()
            myTurn = False
      else:
        closest_fire_point = None
        min_distance = 100
        for fire_point in self.model.firePoints:
          distance = self.dijkstra(self.pos, fire_point.pos)[1]
          if distance < min_distance:
            min_distance = distance
            closest_fire_point = fire_point
        print(f"Agente {self.unique_id} asignado al fuego en {closest_fire_point.pos}")
        path = self.dijkstra(self.pos, closest_fire_point.pos)[0]
        while(myTurn):
          if path and self.actionPoints >= self.calculateSteps(self.pos, path[0]):
            self.actionPoints -= self.calculateSteps(self.pos, path[0])
            self.model.grid.move_agent(self, path[0])
            path.pop(0)
          elif path and self.actionPoints < self.calculateSteps(self.pos, path[0]):
            if self.actionPoints + 4 > 8:
              self.actionPoints = 8
              myTurn = False
            else:
              self.actionPoints += 4
              myTurn = False
          else:
            print("No hay camino al fuego")
            myTurn = False
      if self.carryState == 1:
        self.point = None


    def dijkstra(self, start, end):
      dijkstraMap = {}
      path = []
      for x in range(self.model.grid.height):
        for y in range(self.model.grid.width):
          dijkstraMap[(y, x)] = {"previousCell": None, "steps": None}
      if start in dijkstraMap:
        dijkstraMap[start]["steps"] = 0
        dijkstraMap[start]["previousCell"] = start
        queue = [start]
        while len(queue) > 0:
          for neighbor in self.model.grid.get_neighborhood(queue[0], moore=False):
            if 0 <= neighbor[0] < self.model.grid.width and 0 <= neighbor[1] < self.model.grid.height:
              if dijkstraMap[neighbor]["steps"] is None:
                dijkstraMap[neighbor]["steps"] = self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]
                dijkstraMap[neighbor]["previousCell"] = queue[0]
                queue.append(neighbor)
              elif (dijkstraMap[neighbor]["steps"] > self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]):
                dijkstraMap[neighbor]["steps"] = (self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"])
                dijkstraMap[neighbor]["previousCell"] = queue[0]
                queue.append(neighbor)
          queue.pop(0)
        cell = end
        while cell != start:
          path.insert(0, cell)
          cell = dijkstraMap[cell]["previousCell"]
        return path, dijkstraMap[end]["steps"]
      else:
        print("No existe esa celda de inicio")
        return path, dijkstraMap[end]["steps"]

    def calculateSteps(self, start, end):
          actionPointsCost = 0
          # print(start, end)
          if 0 <= end[0] < len(self.model.cells) and 0 <= end[1] < len(self.model.cells[0]):
            if start[0] < end[0]:
              if self.model.cells[end[0]][end[1]].up or self.model.cells[start[0]][start[1]].down:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
            elif start[0] > end[0]:
              if self.model.cells[end[0]][end[1]].down or self.model.cells[start[0]][start[1]].up:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
            elif start[1] < end[1]:
              if self.model.cells[end[0]][end[1]].left or self.model.cells[start[0]][start[1]].right:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
            elif start[1] > end[1]:
              if self.model.cells[end[0]][end[1]].right or self.model.cells[start[0]][start[1]].left:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
          actionPointsCost = actionPointsCost + 1 * self.carryState
          return actionPointsCost


In [356]:
class MapModel(Model):
    def __init__(self, num_agents, cells, outSide):
        super().__init__()
        self.savedLifes = 0
        self.outSide = outSide
        self.interestPoints = [cell for row in cells for cell in row if cell.alert != 0]
        self.firePoints = [cell for row in cells for cell in row if cell.fire == 2]
        self.width = 10
        self.height = 8
        self.structural_Damage_Left = 24
        self.num_agents = num_agents
        self.grid = MultiGrid(self.height, self.width, False)
        self.cells = cells
        self.schedule = RandomActivation(self)
        self.running = True
        empty_positions = [(x, y) for x in range(self.height) for y in range(self.width)]
        for i in range(self.num_agents):
          random_pos = self.random.choice(empty_positions)
          a = FiremanAgent(i, self)  # No asignamos punto inicialmente
          empty_positions.remove(random_pos)
          self.schedule.add(a)
          self.grid.place_agent(a, random_pos)
          print(f"Agente {a.unique_id} inicio en posición {a.pos}")

    def step(self):
      minSteps = 100
      closestAgent = None
      for i in range(len(self.interestPoints)):
          print("Puntos de interes", self.interestPoints[i].pos)
      for i in range(len(self.interestPoints)):
        closestAgent = None
        for agent in self.schedule.agents:
          if agent.point is None:
            steps = agent.dijkstra(agent.pos, self.interestPoints[i].pos)[1]
            if steps < minSteps:
              minSteps = steps
              closestAgent = agent.unique_id
        for agent in self.schedule.agents:
          if closestAgent == agent.unique_id:
            agent.point = i
            print(f"Agente {agent.unique_id} asignado al punto de interés {i}")
            minSteps = 100
      self.schedule.step()

    def generateNewInterestPoint(self):
      newInterestPointPosition = [(x, y) for x in range(1, self.height - 1) for y in range(1, self.width - 1)]
      random_pos = self.random.choice(newInterestPointPosition)
      self.cells[random_pos[0]][random_pos[1]].alert = self.random.randint(1, 2)
      return self.cells[random_pos[0]][random_pos[1]]


In [357]:
with open('mapa.txt', 'r') as map:
    text = map.read()

walls = []
for i in range(8):
    for j in range(6):
        new_wall = text[:4]
        walls.append(new_wall)
        text = text[5:]

alerts = []
for i in range(3):
    pos_alert_x = text[0]
    pos_alert_y = text[2]
    pos_alert_state = text[4]
    text = text[6:]
    alerts.append( (pos_alert_x, pos_alert_y, pos_alert_state) )

fires = []
for i in range(10):
    pos_fire_x = text[0]
    pos_fire_y = text[2]
    text = text[4:]
    fires.append( (pos_fire_x, pos_fire_y) )

doors = []
for i in range(8):
    pos_doorA_x = text[0]
    pos_doorA_y = text[2]
    pos_doorB_x = text[4]
    pos_doorB_y = text[6]
    text = text[8:]
    doors.append( ( (pos_doorA_x, pos_doorA_y), (pos_doorB_x, pos_doorB_y) ) )

exits = []
for i in range(4):
    pos_exit_x = text[0]
    pos_exit_y = text[2]
    text = text[4:]
    exits.append( (pos_exit_x, pos_exit_y) )

cells = []
for i in range(6):
    for j in range(8):
        w = walls[0]
        del walls[0]

        c = Cell(i + 1,j + 1,w)
        cells.append(c)

        if (str(i + 1), str(j + 1), 'v') in alerts:
            c.alert = 2
        elif (str(i + 1), str(j + 1), 'f') in alerts:
            c.alert = 1

        if (str(i + 1), str(j + 1)) in fires:
            c.fire = 2

        for d in doors:
            if (str(i + 1), str(j + 1)) == d[0]:
                c.door = d[1]
            elif (str(i + 1), str(j + 1)) == d[1]:
                c.door = d[0]

        if (str(i + 1), str(j + 1)) in exits:
            c.entrance = True

        # print(f"{c.pos}: {c.up} - {c.left} - {c.down} - {c.right}   A: {c.alert}   F: {c.fire}   D: {c.door}    E: {c.entrance}")

new_cells = [
    Cell(0, 0, "0000"),
    Cell(0, 1, "0010"),
    Cell(0, 2, "0010"),
    Cell(0, 3, "0010"),
    Cell(0, 4, "0010"),
    Cell(0, 5, "0010"),
    Cell(0, 6, "0010"),
    Cell(0, 7, "0010"),
    Cell(0, 8, "0010"),
    Cell(0, 9, "0000"),
]
outSide = new_cells
cells = new_cells + cells
for i in range (1, 7):
  c = Cell(i, 0, "0001")
  cells.insert(i * 10, c)
  outSide.append(c)
  c = Cell(i, 9, "0100")
  cells.insert((i * 10) + 9, c)
  outSide.append(c)
new_cells = [
    Cell(7, 0, "0000"),
    Cell(7, 1, "1000"),
    Cell(7, 2, "1000"),
    Cell(7, 3, "1000"),
    Cell(7, 4, "1000"),
    Cell(7, 5, "1000"),
    Cell(7, 6, "1000"),
    Cell(7, 7, "1000"),
    Cell(7, 8, "1000"),
    Cell(7, 9, "0000"),
]
outSide = outSide + new_cells
cells = cells + new_cells
map = [[None for _ in range(10)] for _ in range(8)]
for cell in cells:
  y, x = cell.pos
  map[y][x] = cell
  print(f"{map[y][x].pos}: {map[y][x].up} - {map[y][x].left} - {map[y][x].down} - {map[y][x].right}   A: {map[y][x].alert}   F: {map[y][x].fire}   D: {map[y][x].door}    E: {map[y][x].entrance}")


(0, 0): False - False - False - False   A: 0   F: 0   D: []    E: False
(0, 1): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 2): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 3): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 4): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 5): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 6): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 7): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 8): False - False - True - False   A: 0   F: 0   D: []    E: False
(0, 9): False - False - False - False   A: 0   F: 0   D: []    E: False
(1, 0): False - False - False - True   A: 0   F: 0   D: []    E: False
(1, 1): True - True - False - False   A: 0   F: 0   D: []    E: False
(1, 2): True - False - False - False   A: 0   F: 0   D: []    E: False
(1, 3): True - False - False - True   A: 0   F: 0   D: ('1', '4')    E: Fals

In [359]:
model = MapModel(2, map, outSide)
for i in range(25):  # Cambia 10 por el número de pasos que desees
  model.step()
  for agent in model.schedule.agents:
    print(f"Agente {agent.unique_id} en posición {agent.pos}")


Agente 0 inicio en posición (6, 5)
Agente 1 inicio en posición (2, 5)
Puntos de interes (1, 5)
Puntos de interes (3, 7)
Puntos de interes (5, 8)
Agente 1 asignado al punto de interés 0
Agente 0 asignado al punto de interés 1
Agente 0 en posición (5, 5)
Agente 1 en posición (1, 5)
Puntos de interes (6, 7)
Puntos de interes (3, 7)
Puntos de interes (5, 8)
Puntos de interes (0, 5)
Agente 0 asignado al punto de interés 0
Agente 0 en posición (6, 6)
Agente 1 en posición (1, 5)
Puntos de interes (6, 7)
Puntos de interes (3, 7)
Puntos de interes (5, 8)
Puntos de interes (0, 5)
Agente 0 asignado al punto de interés 0
Falsa alarma
Vida salvada
Agente 0 en posición (6, 7)
Agente 1 en posición (0, 5)
Puntos de interes (4, 8)
Puntos de interes (3, 7)
Puntos de interes (5, 8)
Agente 0 asignado al punto de interés 0
Agente 1 asignado al punto de interés 1
Agente 1 en posición (0, 6)
Agente 0 en posición (5, 7)
Puntos de interes (4, 8)
Puntos de interes (3, 7)
Puntos de interes (5, 8)
Agente 0 asigna